Documents

Langchain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content
- metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note than an individual Document object often represents a chunk of a larger document

In [4]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Dogs are loyal and friendly animals, often kept as pets.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Cats are independent and curious creatures, popular as house pets.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Fish are aquatic animals that come in many varieties and are common in home aquariums.", metadata={"source": "fish-pets-doc"}),
    Document(page_content="Parrots are colorful birds known for their ability to mimic sounds and words.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Hamsters are small rodents that are easy to care for and often kept as pets.", metadata={"source": "rodent-pets-doc"}),
    Document(page_content="Turtles are reptiles that can live both in water and on land, and are kept as exotic pets.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Rabbits are gentle mammals that enjoy social interaction and are popular pets.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Goldfish are a type of freshwater fish, known for their bright colors and easy maintenance.", metadata={"source": "fish-pets-doc"}),
]

In [20]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# Vector Stores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)


In [7]:
vectorstore.similarity_search("Which pets are fish?")

[Document(id='06bd31e1-0f89-4d39-85dd-5eef3508e084', metadata={'source': 'fish-pets-doc'}, page_content='Fish are aquatic animals that come in many varieties and are common in home aquariums.'),
 Document(id='a321d95a-d04b-4544-9690-985bb0bf1692', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are a type of freshwater fish, known for their bright colors and easy maintenance.'),
 Document(id='45a7afb4-ef83-4328-9607-52db52c25ead', metadata={'source': 'reptile-pets-doc'}, page_content='Turtles are reptiles that can live both in water and on land, and are kept as exotic pets.'),
 Document(id='d2c7af07-9110-4f48-9f3a-86007e3c5cce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent and curious creatures, popular as house pets.')]

In [8]:
# Async Query
await vectorstore.asimilarity_search("Which pets are fish?")

[Document(id='06bd31e1-0f89-4d39-85dd-5eef3508e084', metadata={'source': 'fish-pets-doc'}, page_content='Fish are aquatic animals that come in many varieties and are common in home aquariums.'),
 Document(id='a321d95a-d04b-4544-9690-985bb0bf1692', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are a type of freshwater fish, known for their bright colors and easy maintenance.'),
 Document(id='45a7afb4-ef83-4328-9607-52db52c25ead', metadata={'source': 'reptile-pets-doc'}, page_content='Turtles are reptiles that can live both in water and on land, and are kept as exotic pets.'),
 Document(id='d2c7af07-9110-4f48-9f3a-86007e3c5cce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent and curious creatures, popular as house pets.')]

In [9]:
vectorstore.similarity_search_with_score("Which pets are fish?")

[(Document(id='06bd31e1-0f89-4d39-85dd-5eef3508e084', metadata={'source': 'fish-pets-doc'}, page_content='Fish are aquatic animals that come in many varieties and are common in home aquariums.'),
  0.7524722814559937),
 (Document(id='a321d95a-d04b-4544-9690-985bb0bf1692', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are a type of freshwater fish, known for their bright colors and easy maintenance.'),
  1.1597652435302734),
 (Document(id='45a7afb4-ef83-4328-9607-52db52c25ead', metadata={'source': 'reptile-pets-doc'}, page_content='Turtles are reptiles that can live both in water and on land, and are kept as exotic pets.'),
  1.1644378900527954),
 (Document(id='d2c7af07-9110-4f48-9f3a-86007e3c5cce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent and curious creatures, popular as house pets.'),
  1.210528016090393)]

# Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into Langchain Expression Language chains.

Langchain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:



In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Get top result for each query of batch
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["Cat", "Dog"])

[[Document(id='d2c7af07-9110-4f48-9f3a-86007e3c5cce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent and curious creatures, popular as house pets.')],
 [Document(id='e4123127-813f-4133-8ad7-7852e158a8fc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are loyal and friendly animals, often kept as pets.')]]

Vectorstores implement an "as_retriever" method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parametrize them. For instance, we can replicate the above with the following:

In [14]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 1}
)

retriever.batch(["Cat", "Dog"])

[[Document(id='d2c7af07-9110-4f48-9f3a-86007e3c5cce', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent and curious creatures, popular as house pets.')],
 [Document(id='e4123127-813f-4133-8ad7-7852e158a8fc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are loyal and friendly animals, often kept as pets.')]]

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:

{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("human", message)]
)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
response = rag_chain.invoke("Fish?")
print(response.content)

Fish are aquatic animals that come in many varieties and are common in home aquariums.  

